In [9]:
import pandas as pd
import numpy as np
data = pd.read_excel('每日行情数据统计_600567.xlsx')
data=data.loc[:,['交易日期','收盘价(元)']]
data=data.rename(columns={'交易日期': 'Date', '收盘价(元)': 'close'})
data=data.head(243)





In [20]:
#计算波动率（每日收益率的波动率+期望）
#data是一个pandas的DataFrame，有两列，第一列是日期，第二列是收盘价
#返回一个float，波动率
#波动率在随机游走模型中必须是一个正值
import pandas as pd
import numpy as np    
def volatility(data):
    #计算每日收益率
    data['daily_return'] = data['close'].pct_change()
    #计算波动率
    return data['daily_return'].std()
def expected_daily_return(data):
    data['daily_return'] = data['close'].pct_change()
    data.dropna(inplace=True)
    expected_daily_return = data['daily_return'].mean()
    print(data['daily_return'].tail(10))
    return expected_daily_return

print(volatility(data))
print(expected_daily_return(data))

0.015737529250406435
233    0.004444
234   -0.004425
235    0.017778
236   -0.008734
237    0.013216
238   -0.004348
239   -0.026201
240    0.004484
241   -0.004464
242   -0.004484
Name: daily_return, dtype: float64
0.0016126296519820256


In [ ]:
#1、连续120个交易日通过本所交易系统实现的累计股票成交量低于500万股；



In [15]:
#2、连续20个交易日的每日股票收盘价均低于1元；
#未来有一天股价低于1元的概率
#vol 表示股价的波动率。(t ** 0.5) 表示时间 t 的平方根，这是因为在随机游走模型中，股价的波动随着时间的平方根增长。
from scipy import stats
import numpy as np

vol=volatility(data)
t=300
s=1.47
p = stats.norm.cdf((np.log(1) - np.log(s)) / (vol * (t ** 0.5)))
p

0.10896189365719494

In [16]:
#2、连续20个交易日的每日股票收盘价均低于1元；
#蒙特卡洛
import numpy as np
import pandas as pd

def simulate_stock_paths(S0, mu, sigma, T, dt, num_simulations):
    """
    模拟股票价格路径
    S0: 初始股价
    mu: 每日收益率的期望值
    sigma: 每日收益率的波动率
    T: 总时间（天）
    dt: 时间步长（天）
    num_simulations: 模拟路径数量
    """
    num_steps = int(T / dt)
    paths = np.zeros((num_simulations, num_steps))
    paths[:, 0] = S0

    for t in range(1, num_steps):
        z = np.random.standard_normal(num_simulations)
        paths[:, t] = paths[:, t - 1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
    
    return paths

def estimate_probability(S0, mu, sigma, T, dt, num_simulations, threshold, duration):
    """
    估计股价连续低于某一水平的概率
    S0: 初始股价
    mu: 每日收益率的期望值
    sigma: 每日收益率的波动率
    T: 总时间（天）
    dt: 时间步长（天）
    num_simulations: 模拟路径数量
    threshold: 股价阈值（1元）
    duration: 连续低于阈值的天数（20天）
    """
    paths = simulate_stock_paths(S0, mu, sigma, T, dt, num_simulations)
    count = 0

    for path in paths:
        for i in range(len(path) - duration + 1):
            if np.all(path[i:i+duration] < threshold):
                count += 1
                break
    
    probability = count / num_simulations
    return probability

# 参数设置
S0 = 1.47  # 初始股价
mu = expected_daily_return(data) # 每日收益率的期望值
sigma = volatility(data)  # 每日收益率的波动率
T = 300  # 总时间（天）
dt = 1  # 时间步长（天）
num_simulations = 10000  # 模拟路径数量
threshold = 1  # 股价阈值
duration = 20  # 连续低于阈值的天数

# 估计概率
probability = estimate_probability(S0, mu, sigma, T, dt, num_simulations, threshold, duration)
print(f"股价连续 {duration} 天低于 {threshold} 元的概率: {probability:}")


股价连续 20 天低于 1 元的概率: 0.0019


In [ ]:
#3、连续20个交易日的每日股票收盘总市值均低于5亿元；


In [ ]:
#4、公司连续二十个交易日股东人数均少于 2000 人；